<a href="https://colab.research.google.com/github/ibacaraujo/pysyft-learning/blob/master/Part_04_Federated_Learning_via_Model_Averaging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 04. Federated Learning via Model Averaging

In [1]:
!pip install tf-encrypted

! URL="https://github.com/openmined/PySyft.git" && FOLDER="PySyft" && if [ ! -d $FOLDER ]; then git clone -b dev --single-branch $URL; else (cd $FOLDER && git pull $URL && cd ..); fi;

!cd PySyft; python setup.py install  > /dev/null

import os
import sys
module_path = os.path.abspath(os.path.join('./PySyft'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
!pip install --upgrade --force-reinstall lz4
!pip install --upgrade --force-reinstall websocket
!pip install --upgrade --force-reinstall websockets
!pip install --upgrade --force-reinstall zstd

     |████████████████████████████████| 2.7MB 4.8MB/s 
     |████████████████████████████████| 276kB 30.6MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.3-cp36-cp36m-linux_x86_64.whl size=44229 sha256=ee8b8fdab562e74f9ea2d943b01ddcd8072e6b83c645ef9474ba6ce8470c87cb
  Stored in directory: /root/.cache/pip/wheels/e4/76/4d/a95b8dd7b452b69e8ed4f68b69e1b55e12c9c9624dd962b191
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Cloning into 'PySyft'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 31660 (delta 0), reused 1 (delta 0), pack-reused 31659
Receiving objects: 100% (31660/31660), 33.65 MiB | 20.29 MiB/s, done.
Resolving deltas: 100% (21345/21345), done.
zip_safe flag not set; analyzing archive contents...
zip_safe flag not set; analyzing archive contents...
__pycache__.zstd.cpython-36: module references __file__
     |███████████████████████

In [45]:
import torch
import syft as sy
import copy
from torch import nn, optim

hook = sy.TorchHook(torch)

## Step 1. Create Data Owners

In [0]:
# create a couple workers
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

# A Toy Dataset
data = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1.]], requires_grad=True)
target = torch.tensor([[0], [0], [1], [1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

## Step 2. Create Our Model

In [0]:
# Initialize a Toy Model
model = nn.Linear(2,1)

## Step 3. Send a Copy of the Model to Alice and Bob

In [0]:
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

## Step 4. Train Bob's and Alice's Model (in parallel)

In [49]:
for i in range(10):

  # Train Bob's Model
  bobs_opt.zero_grad()
  bobs_pred = bobs_model(bobs_data)
  bobs_loss = ((bobs_pred - bobs_target)**2).sum()
  bobs_loss.backward()

  bobs_opt.step()
  bobs_loss = bobs_loss.get().data

  # Train Alice's Model
  alices_opt.zero_grad()
  alices_pred = alices_model(alices_data)
  alices_loss = ((alices_pred - alices_target)**2).sum()
  alices_loss.backward()

  alices_opt.step()
  alices_loss = alices_loss.get().data

  print("Bob: " + str(bobs_loss) + " Alice: " + str(alices_loss))

Bob: tensor(1.9910) Alice: tensor(0.5599)
Bob: tensor(0.4538) Alice: tensor(0.1740)
Bob: tensor(0.1046) Alice: tensor(0.1426)
Bob: tensor(0.0251) Alice: tensor(0.1186)
Bob: tensor(0.0069) Alice: tensor(0.0987)
Bob: tensor(0.0026) Alice: tensor(0.0822)
Bob: tensor(0.0014) Alice: tensor(0.0684)
Bob: tensor(0.0011) Alice: tensor(0.0569)
Bob: tensor(0.0009) Alice: tensor(0.0474)
Bob: tensor(0.0007) Alice: tensor(0.0394)


## Step 5. Send Both Updated Models to a Secure Worker

In [0]:
alices_model.move(secure_worker)

In [0]:
bobs_model.move(secure_worker)

## Step 6. Average the Models

In [0]:
with torch.no_grad():
  model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
  model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())

## Rinse and Repeat

In [53]:
iterations = 10
worker_iters = 5

for a_iter in range(iterations):
    
  bobs_model = model.copy().send(bob)
  alices_model = model.copy().send(alice)

  bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
  alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

  for wi in range(worker_iters):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).sum()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).sum()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    
  alices_model.move(secure_worker)
  bobs_model.move(secure_worker)
  with torch.no_grad():
      model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
      model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
  print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.0051) Alice:tensor(0.0244)
Bob:tensor(0.0073) Alice:tensor(0.0116)
Bob:tensor(0.0084) Alice:tensor(0.0057)
Bob:tensor(0.0083) Alice:tensor(0.0029)
Bob:tensor(0.0076) Alice:tensor(0.0016)
Bob:tensor(0.0066) Alice:tensor(0.0009)
Bob:tensor(0.0056) Alice:tensor(0.0005)
Bob:tensor(0.0046) Alice:tensor(0.0003)
Bob:tensor(0.0037) Alice:tensor(0.0002)
Bob:tensor(0.0029) Alice:tensor(0.0001)


In [0]:
preds = model(data)
loss = ((preds - target)**2).sum()

In [55]:
print(preds)
print(target)
print(loss.data)

tensor([[0.1575],
        [0.1317],
        [0.8333],
        [0.8075]], grad_fn=<AddmmBackward>)
tensor([[0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)
tensor(0.1070)
